In [3]:
# 미리 다운로드 받아야 할 파일
# bvlc_googlenet.caffemodel,
# deploy.prototxt
# classification_classes_ILSVRC2012.txt

import os
import sys
import cv2
import numpy as np

#입력 영상 지정  , "scooter.jpg"로도 바꿔 테스트해볼 것
filename = "../images/space_shuttle.jpg"
# filename = "../images/scooter.jpg"
img = cv2.imread(filename)

# Caffe 모델 불러오기
#다운로드 링크 : http://dl.caffe.berkeleyvision.org/bvlc_googlenet.caffemodel
model = "../models/googlenet/bvlc_googlenet.caffemodel"
#다운로드 링크 : https://github.com/BVLC/caffe/blob/master/models/bvlc_googlenet/deploy.prototxt
config = "../models/googlenet/deploy.prototxt"

files = [model, config]

#설정파일 있는지 체크
for f in files:
    if os.path.isfile(f) is False:
        print("모델 혹은 설정파일이 없습니다 : "+f)
        sys.exit()

net = cv2.dnn.readNet(model, config)  # 모델 불러오기

if net.empty():
    print("Network load failed!")
    sys.exit()

# 클래스 이름 불러오기; classNames에 1000개의 name을 list로 저장
classNames = None
with open("../models/googlenet/classification_classes_ILSVRC2012.txt", "rt") as f:
    classNames = f.read().rstrip("\n").split("\n")

# 추론
blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123))  #이미지 크기, 컬러 평균값 지정
net.setInput(blob)  #입력 설정
prob = net.forward()  #추론 - prob : 1행 1000열

# 추론 결과 확인 & 화면 출력
out = prob[0]   # 첫 번째 결과만 슬라이싱
classId = np.argmax(out)  #가장 높은 확률값의 행번호 반환
confidence = out[classId]  #확률값 저장

#클래스네임과 확률값 출력
text = f"{classNames[classId]} ({confidence * 100:4.2f}%)"  
cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow("img", img)
cv2.waitKey()
cv2.destroyAllWindows()
